# Auto ML Model

In [1]:
from h2o.automl import H2OAutoML
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

/Users/wonjaelee/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "23.0.1" 2024-10-15; OpenJDK Runtime Environment Homebrew (build 23.0.1); OpenJDK 64-Bit Server VM Homebrew (build 23.0.1, mixed mode, sharing)
  Starting server from /Users/wonjaelee/Library/Python/3.9/lib/python/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/zb/wwx9tqtd75dgcqk6pxlq3w3w0000gn/T/tmpz2lyoe5p
  JVM stdout: /var/folders/zb/wwx9tqtd75dgcqk6pxlq3w3w0000gn/T/tmpz2lyoe5p/h2o_wonjaelee_started_from_python.out
  JVM stderr: /var/folders/zb/wwx9tqtd75dgcqk6pxlq3w3w0000gn/T/tmpz2lyoe5p/h2o_wonjaelee_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,H2O_from_python_wonjaelee_7zntx6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.984 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [5]:
import pandas as pd
data = pd.read_csv('processed_data.csv')
df_f = pd.get_dummies(data, columns=['name','neighbourhood_group','room_type','license'], drop_first=True)

In [6]:
df_f.shape

(20666, 54)

In [7]:
h2o_frame = h2o.H2OFrame(df_f)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
train, test = h2o_frame.split_frame(ratios=[0.8], seed=1234)

In [9]:
x = train.columns
y = "price-transform"
x.remove(y)

In [10]:
aml = H2OAutoML(max_models=20, seed=123, nfolds = 2)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
13:17:38.551: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),13/20
# GBM base models (used / total),10/10
# DRF base models (used / total),2/2
# DeepLearning base models (used / total),1/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,2
Metalearner fold_column,None


In [11]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20241207_131738,0.136122,0.0185292,0.103407,0.0344391,0.0185292
StackedEnsemble_BestOfFamily_1_AutoML_1_20241207_131738,0.137426,0.0188858,0.104105,0.034758,0.0188858
GBM_3_AutoML_1_20241207_131738,0.138176,0.0190926,0.104987,0.0349536,0.0190926
GBM_5_AutoML_1_20241207_131738,0.13833,0.0191352,0.105954,0.0350115,0.0191352
GBM_2_AutoML_1_20241207_131738,0.138591,0.0192074,0.105799,0.0350607,0.0192074
GBM_4_AutoML_1_20241207_131738,0.139178,0.0193706,0.105454,0.0352061,0.0193706
GBM_grid_1_AutoML_1_20241207_131738_model_1,0.139441,0.0194438,0.106232,0.0352661,0.0194438
GBM_grid_1_AutoML_1_20241207_131738_model_3,0.139856,0.0195596,0.106506,0.035372,0.0195596
GBM_grid_1_AutoML_1_20241207_131738_model_4,0.139877,0.0195655,0.106633,0.0354011,0.0195655
GBM_1_AutoML_1_20241207_131738,0.140198,0.0196553,0.107308,0.0355011,0.0196553


In [12]:
m = aml.leader

# Evaluate the best model's performance on the test set
performance = m.model_performance(test)

# Get the R² value
r2_value = performance.r2()
print(f"R² value: {r2_value}")
# Get the RMSE value
rmse_value = performance.rmse()
print(f"RMSE value: {rmse_value}")
# Get the MAE value
mae_value = performance.mae()
print(f"MAE value: {mae_value}")

R² value: 0.6691702655730898
RMSE value: 0.13614774703571353
MAE value: 0.10237857299223632


In [16]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
tpotaml = TPOTRegressor(generations=5, population_size=20, random_state=42)

In [17]:
X = df_f.drop(columns=['price-transform'])
y = df_f['price-transform']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
tpotaml.fit(X_train, y_train)

TPOTRegressor(generations=5, population_size=20, random_state=42)

In [20]:
tpotaml.export('airnbnb_best_model.py')

In [21]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = df_f
features = tpot_data.drop('price-transform', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['price-transform'], random_state=42)

# Average CV score on the training set was: -0.020139319297559362
exported_pipeline = RandomForestRegressor(bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=4, min_samples_split=10, n_estimators=100)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

In [22]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [23]:
validation_scores = cross_val_score(exported_pipeline, X_train, y_train, cv=5)  # 5-fold cross-validation
print("Mean Validation Score:", validation_scores.mean())

mae_scores = cross_val_score(exported_pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
mse_scores = cross_val_score(exported_pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Convert to positive values
mae_scores = -mae_scores
rmse_scores = np.sqrt(-mse_scores)

# Calculate mean and standard deviation
print ("Validation MAE" , np.mean(mae_scores))
print ("Validation RMSE" , np.mean(rmse_scores))

Mean Validation Score: 0.6425046341646365
Validation MAE 0.10709865275271531
Validation RMSE 0.14184758074291026


In [24]:
print("RMSE",np.sqrt(mean_squared_error(testing_target,results)))
print("R2", r2_score(testing_target,results))
print("MAE", mean_absolute_error(testing_target,results))

RMSE 0.13561763839731802
R2 0.6653091639601738
MAE 0.10309085849632507
